In [1]:
# general imports
import numpy as np
import pandas as pd
import radioactivedecay as rd
from pathlib import Path

# TARDIS imports for the gamma-ray code
from tardis.io.atom_data import AtomData
from tardis.model import SimulationState
from tardis.plasma.base import BasePlasma
from tardis.io.configuration import config_reader
from tardis.io.atom_data.util import download_atom_data
from tardis.energy_input.energy_source import get_nuclear_lines_database
from tardis.energy_input.gamma_ray_channel import time_evolve_cumulative_decays, create_isotope_dicts, create_inventories_dict, calculate_total_decays
from tardis.plasma.properties import Density, Abundance, IsotopeAbundance, \
    IsotopeNumberDensity, AtomicData, AtomicMass, IsotopeMass, NumberDensity, \
    SelectedAtoms

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
# Download the atom data file from tardis-refdata repo to run this cell.
atom_data_file = 'kurucz_cd23_chianti_H_He.h5'
atom_data = AtomData.from_hdf(atom_data_file)

INFO:tardis.io.atom_data.base:Reading Atom Data with: UUID = 6f7b09e887a311e7a06b246e96350010 MD5  = 864f1753714343c41f99cb065710cace 
INFO:tardis.io.atom_data.base:Non provided Atomic Data: synpp_refs, photoionization_data, yg_data, two_photon_data, linelist
INFO:tardis.io.atom_data.base:Non provided Atomic Data: synpp_refs, photoionization_data, yg_data, two_photon_data, linelist


In [3]:
# Read the config file and create a configuration object
config = config_reader.Configuration.from_yaml("tardis_configv1_density_exponential_nebular.yml")

In [4]:
# Create the model
model = SimulationState.from_csvy(config, atom_data)

INFO:tardis.model.matter.decay:Decaying abundances for 2600540.0352 seconds
INFO:tardis.model.matter.decay:Decaying abundances for 2600540.0352 seconds
/Users/anirbandutta/Software/tardis/tardis/model/matter/composition.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '7.961942738176252e-23' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  isotope_mass_df[isotope_tuple] = isotope_mass


In [5]:
gamma_ray_lines = get_nuclear_lines_database(atom_data_file)

In [6]:
raw_isotope_mass_fraction = model.composition.raw_isotope_abundance

In [7]:
# Construct the Plasma

input = [Density, Abundance, IsotopeAbundance, AtomicData, AtomicMass, IsotopeNumberDensity, NumberDensity, SelectedAtoms, IsotopeMass]

plasma = BasePlasma(plasma_properties=input, density = model.density, 
                    abundance=model.abundance, isotope_abundance=model.composition.raw_isotope_abundance,
                    atomic_data = atom_data)

In [8]:
shell_masses = model.volume * model.density

In [9]:
time_array = np.linspace(0.01, 80, 50)

In [10]:
time_evolved_decay_df = time_evolve_cumulative_decays(raw_isotope_mass_fraction, shell_masses, gamma_ray_lines, time_array)

In [12]:
time_evolved_decay_df.shape

(937200, 8)

In [11]:
time_evolved_decay_df.head(30)

number_of_decays decay_mode radiation  \
time shell_number isotope                                          
0.01 0            Fe52         5.098882e+30         EC        bp   
                  Fe52         5.098882e+30         EC     bp av   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC        bp   
                  Fe52         5.098882e+30         EC     bp av   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         e   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   
                  Fe52         5.098882e+30         EC         g   

                           radiation_energy_keV  radiation_intensity  \
time shell_number isotope                                              
0.01 0            Fe52               340.000000            55.490000   
                  Fe52               340.000001            55.490000   
                  Fe52                 5.190000            26.900000   
                  Fe52                 0.610000            62.500000   
                  Fe52               162.149000             0.699000   
                  Fe52               371.209000             0.058500   
                  Fe52               167.919000             0.067300   
                  Fe52               376.979000             0.006280   
                  Fe52               168.688000             0.009100   
                  Fe52               377.748000             0.000846   
                  Fe52               168.688000             0.000433   
                  Fe52               511.000000           110.980000   
                  Fe52                 5.899000             7.300000   
                  Fe52                 5.888000             3.690000   
                  Fe52                 6.490000             0.850000   
                  Fe52                 6.490000             0.433000   
                  Fe52                 0.640000             0.300000   
                  Fe52               168.688000            99.000000   
                  Fe52               377.748000             1.640000   
                  Fe52              1039.928000             0.095000   
                  Fe52              2058.100000            99.620000   
                  Fe52              2058.000000            99.620000   
                  Fe52                 5.190000             0.229900   
                  Fe52                 0.610000   